In [42]:
import pandas as pd
import numpy as np
import joblib
import os
from glob import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import roc_auc_score
from itertools import product
import random


# 讀取並合併 TSFEL 資料

In [ ]:
# 讀取所有 train 資料夾底下的 CSV 檔案
csv_files = glob(os.path.join('TSFEL/train', '*.csv'))
# 合併所有 CSV 成為一份完整資料
train_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
print(f"📁 成功合併 {len(csv_files)} 個檔案，共 {len(train_df)} 筆資料")

# === 1. 合併 test/test/ 下的所有 CSV ===
test_files = glob(os.path.join('TSFEL', 'test', '*.csv'))
test_df = pd.concat([pd.read_csv(f) for f in test_files], ignore_index=True)
print(f"📁 合併完成，共 {len(test_files)} 個檔案，總筆數：{len(test_df)}")

📁 成功合併 10 個檔案，共 52588 筆資料
📁 合併完成，共 10 個檔案，總筆數：38014


In [44]:
a = [i for i in train_df.columns if i not in test_df.columns]
a

['player_id', 'gender', 'hold racket handed', 'play years', 'level']

In [45]:
# ==== 1. 定義設定 ====
drop_cols = [
    'cut_point_start', 'cut_point_end', 'unique_id', 'player_id',
    'gender', 'hold racket handed', 'play years', 'level',
    'Gx_Kurtosis', 'Gx_Skewness', 'Gz_Kurtosis', 'Gz_Skewness'
]
mode_encoder_path = 'mode_encoder.pkl'

In [ ]:
# 若您已分割好 train_df, test_df，直接使用也可
train_df['gender_binary'] = train_df['gender'].map({1: 0, 2: 1})
train_df['hold_racket_binary'] = train_df['hold racket handed'].map({1: 0, 2: 1})

# 儲存 unique_id 供預測時使用
unique_ids = test_df['unique_id']


In [47]:
# ==== 3. 類別欄位編碼（mode）====
le_mode = LabelEncoder()
train_df['mode'] = le_mode.fit_transform(train_df['mode'])
test_df['mode'] = le_mode.transform(test_df['mode'])
# ==== 類別欄位處理：改用 category dtype ====
train_df['mode'] = train_df['mode'].astype('category')
test_df['mode'] = test_df['mode'].astype('category')

joblib.dump(le_mode, mode_encoder_path)

['mode_encoder.pkl']

# 定義各種使用到的函式
- 切分訓練、驗證資料
- 隨機搜索方法
- SMOTE 方法
- 訓練 XGBoost 並預測測試資料

In [52]:
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from itertools import product
from collections import Counter
import pandas as pd
import numpy as np

def stratified_player_split(df, y, test_size=0.2, random_state=42):
    player_groups = df.groupby('player_id')
    player_ids = list(player_groups.groups.keys())

    y_per_player = []
    for pid in player_ids:
        labels = y[df['player_id'] == pid]
        label = Counter(labels).most_common(1)[0][0]
        y_per_player.append(label)

    y_array = np.array(y_per_player)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    train_idx, valid_idx = next(sss.split(np.zeros(len(player_ids)), y_array))

    train_players = [player_ids[i] for i in train_idx]
    valid_players = [player_ids[i] for i in valid_idx]

    train_mask = df['player_id'].isin(train_players)
    valid_mask = df['player_id'].isin(valid_players)

    print("\n📊 每個類別在訓練與驗證集的 player 數：")
    def show_distribution(pids, title):
        dist = Counter([Counter(y[df['player_id'] == pid]).most_common(1)[0][0] for pid in pids])
        print(f"🔹 {title}：{dict(dist)}")
    show_distribution(train_players, "訓練集")
    show_distribution(valid_players, "驗證集")

    print(f"👤 驗證集 player_id：{valid_players}")  # 👈 加在這裡

    return train_mask, valid_mask

def manual_random_search(X_all, y_all, param_grid, num_classes, n_iter=20, seed=42, fold_count=3, verbose=True):
    from xgboost import XGBClassifier
    from sklearn.metrics import roc_auc_score

    all_combos = list(product(*param_grid.values()))
    random.seed(seed)
    sampled_combos = random.sample(all_combos, min(n_iter, len(all_combos)))

    best_score = -1
    best_model = None
    best_params = None

    print(f"\n🔍 開始搜尋共 {len(sampled_combos)} 組參數，每組做 {fold_count}-Fold 驗證\n" + "-"*60)

    for i, combo in enumerate(sampled_combos):
        params = dict(zip(param_grid.keys(), combo))
        scores = []

        print(f"\n🧪 測試第 {i+1}/{len(sampled_combos)} 組參數：{params}")

        for fold in range(fold_count):
            train_mask, valid_mask = stratified_player_split(train_df, y_all, random_state=seed + fold)
            X_train, y_train = X_all[train_mask], y_all[train_mask]
            X_valid, y_valid = X_all[valid_mask], y_all[valid_mask]

            model = XGBClassifier(
                tree_method='hist',
                enable_categorical=True,
                eval_metric='logloss',
                random_state=42,
                **params
            )
            model.fit(X_train, y_train)
            y_pred_proba = model.predict_proba(X_valid)

            if num_classes == 2:
                score = roc_auc_score(y_valid, y_pred_proba[:, 1])
            else:
                score = roc_auc_score(y_valid, y_pred_proba, multi_class='ovr', average='macro')

            scores.append(score)
            if verbose:
                print(f"  ➤ Fold {fold+1} ROC AUC: {score:.4f}")

        avg_score = np.mean(scores)
        print(f"✅ 平均 ROC AUC: {avg_score:.4f}")

        if avg_score > best_score:
            best_score = avg_score
            best_model = model
            best_params = params
            print(f"🎯 新最佳模型！已更新\n{'='*60}")

    print(f"\n🏁 最佳 ROC AUC: {best_score:.4f} 對應參數：{best_params}")
    return best_model, best_params, best_score

def apply_smote(X, y, smote_ratio):
    counter = Counter(y)
    max_count = max(counter.values())
    strategy = {
        cls: int(max_count * smote_ratio)
        for cls, count in counter.items()
        if int(max_count * smote_ratio) > count
    }
    if strategy:
        print(f"🧪 SMOTE: {strategy}")
        smote = SMOTE(sampling_strategy=strategy, random_state=42)
        return smote.fit_resample(X, y)
    return X, y

def train_and_predict_xgb(
    y_name,
    return_model=False,
    smote_ratio=None,
    cv_mode='single',     # 'single' 或 'manual_kfold'
    fold_count=3,
    n_iter=20
):
    print(f"\n🚀 開始訓練任務：{y_name}（cv_mode={cv_mode}）")

    # === y 處理 ===
    if y_name == 'gender':
        y_all = train_df['gender'].map({1: 0, 2: 1})
        num_classes = 2
    elif y_name == 'hold racket handed':
        y_all = train_df['hold racket handed'].map({1: 0, 2: 1})
        num_classes = 2
    elif y_name == 'level':
        y_all = train_df['level'].map({2: 0, 3: 1, 4: 2, 5: 3})
        num_classes = 4
    else:
        y_all = train_df[y_name]
        num_classes = len(set(y_all))

    # === 特徵處理 ===
    drop_y = {'gender', 'hold racket handed', 'play years', 'level'} - {y_name}
    drop_all = set(drop_cols).union(drop_y)
    common_cols = [c for c in train_df.columns if c not in drop_all and c in test_df.columns]
    X_all = train_df[common_cols]
    X_test = test_df[common_cols]

    # ✅ 類別欄位需為 category dtype，並保留 enable_categorical
    assert str(X_all.dtypes[X_all.select_dtypes(include='category').columns[0]]) == "category"

    param_grid = {
        'n_estimators': [100, 200, 300, 500],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'gamma': [0, 1, 5],
        'scale_pos_weight': [1, 5, 10, 20] if num_classes == 2 else [1]
    }

    # ✅ 找出最佳參數與模型
    model, best_params, best_score = manual_random_search(
        X_all, y_all,
        param_grid=param_grid,
        num_classes=num_classes,
        n_iter=n_iter,
        seed=42,
        fold_count=fold_count
    )

    print(f"\n🏆 最佳 ROC AUC: {best_score:.4f}")
    print(f"🎯 使用最佳參數重新訓練完整模型並預測測試集\n{'='*60}")

    # ✅ 使用最佳參數重新訓練完整模型（不分訓練/驗證集）
    model = XGBClassifier(
        tree_method='hist',
        enable_categorical=True,
        eval_metric='logloss',
        random_state=42,
        **best_params
    )
    model.fit(X_all, y_all)

    # ✅ 預測測試集
    y_proba = model.predict_proba(X_test)
    if num_classes == 2:
        df_pred = pd.DataFrame({y_name: y_proba[:, 0]})
    else:
        df_pred = pd.DataFrame(y_proba, columns=[f"{y_name}_{i}" for i in range(num_classes)])

    return (df_pred, model, common_cols) if return_model else df_pred


# 對於四個目標預測測試資料

In [53]:
# === 性別預測（gender）===
df_gender, model_gender, cols_gender = train_and_predict_xgb(
    y_name='gender',
    smote_ratio=1,
    return_model=True,
    cv_mode='manual_kfold',
    fold_count=5,
    n_iter=30
)


🚀 開始訓練任務：gender（cv_mode=manual_kfold）

🔍 開始搜尋共 30 組參數，每組做 5-Fold 驗證
------------------------------------------------------------

🧪 測試第 1/30 組參數：{'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 1, 'scale_pos_weight': 10}

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 27, 1: 6}
🔹 驗證集：{0: 7, 1: 2}
👤 驗證集 player_id：[42, 7, 37, 11, 15, 27, 8, 25, 21]
  ➤ Fold 1 ROC AUC: 0.8672

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 27, 1: 6}
🔹 驗證集：{1: 2, 0: 7}
👤 驗證集 player_id：[25, 34, 22, 17, 5, 18, 1, 40, 26]
  ➤ Fold 2 ROC AUC: 0.8588

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 27, 1: 6}
🔹 驗證集：{1: 2, 0: 7}
👤 驗證集 player_id：[29, 41, 36, 11, 1, 21, 42, 16, 4]
  ➤ Fold 3 ROC AUC: 0.5171

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 27, 1: 6}
🔹 驗證集：{0: 7, 1: 2}
👤 驗證集 player_id：[12, 40, 1, 28, 22, 39, 4, 26, 37]
  ➤ Fold 4 ROC AUC: 0.7794

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 27, 1: 6}
🔹 驗證集：{0: 7, 1: 2}
👤 驗證集 player_id：[37, 29, 35, 24, 12, 19, 38, 6, 20]
  ➤ Fold 5 ROC AUC: 0.687

In [54]:
# === 握拍手預測（hold racket handed）===
df_hold, model_hold, cols_hold = train_and_predict_xgb(
    y_name='hold racket handed',
    smote_ratio=0.7,
    return_model=True,
    cv_mode='manual_kfold',
    fold_count=5,
    n_iter=30
)


🚀 開始訓練任務：hold racket handed（cv_mode=manual_kfold）

🔍 開始搜尋共 30 組參數，每組做 5-Fold 驗證
------------------------------------------------------------

🧪 測試第 1/30 組參數：{'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 1, 'scale_pos_weight': 10}

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 27, 1: 6}
🔹 驗證集：{0: 8, 1: 1}
👤 驗證集 player_id：[10, 23, 18, 42, 11, 35, 25, 34, 31]
  ➤ Fold 1 ROC AUC: 0.9992

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 27, 1: 6}
🔹 驗證集：{0: 8, 1: 1}
👤 驗證集 player_id：[24, 27, 36, 33, 26, 1, 3, 22, 21]
  ➤ Fold 2 ROC AUC: 0.9919

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 27, 1: 6}
🔹 驗證集：{0: 8, 1: 1}
👤 驗證集 player_id：[25, 42, 39, 19, 8, 4, 14, 29, 30]
  ➤ Fold 3 ROC AUC: 0.9983

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 28, 1: 5}
🔹 驗證集：{0: 7, 1: 2}
👤 驗證集 player_id：[36, 37, 26, 39, 27, 38, 4, 41, 8]
  ➤ Fold 4 ROC AUC: 0.9988

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 28, 1: 5}
🔹 驗證集：{0: 7, 1: 2}
👤 驗證集 player_id：[35, 34, 15, 5, 9, 23, 36, 24, 33]
  ➤ Fold 5 R

In [55]:
# === 球齡預測（play years）===
df_play, model_play, cols_play = train_and_predict_xgb(
    y_name='play years',
    smote_ratio=0.7,
    return_model=True,
    cv_mode='manual_kfold',
    fold_count=5,
    n_iter=30
)


🚀 開始訓練任務：play years（cv_mode=manual_kfold）

🔍 開始搜尋共 30 組參數，每組做 5-Fold 驗證
------------------------------------------------------------

🧪 測試第 1/30 組參數：{'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 1, 'scale_pos_weight': 1}

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{1: 13, 0: 6, 2: 14}
🔹 驗證集：{1: 4, 0: 2, 2: 3}
👤 驗證集 player_id：[7, 12, 18, 42, 35, 24, 19, 26, 10]


/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:41:43] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


  ➤ Fold 1 ROC AUC: 0.5964

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{1: 13, 0: 6, 2: 14}
🔹 驗證集：{2: 3, 1: 4, 0: 2}
👤 驗證集 player_id：[13, 40, 4, 19, 18, 6, 17, 3, 41]


/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:42:58] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


  ➤ Fold 2 ROC AUC: 0.6064

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 6, 2: 14, 1: 13}
🔹 驗證集：{2: 3, 1: 4, 0: 2}
👤 驗證集 player_id：[29, 23, 22, 6, 41, 21, 35, 10, 5]


...

/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [23:39:45] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


  ➤ Fold 4 ROC AUC: 0.6207

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{1: 14, 2: 13, 0: 6}
🔹 驗證集：{0: 2, 2: 4, 1: 3}
👤 驗證集 player_id：[31, 26, 36, 15, 30, 42, 3, 17, 4]


/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [23:40:32] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


  ➤ Fold 5 ROC AUC: 0.7299
✅ 平均 ROC AUC: 0.6435

🏁 最佳 ROC AUC: 0.6476 對應參數：{'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'scale_pos_weight': 1}

🏆 最佳 ROC AUC: 0.6476
🎯 使用最佳參數重新訓練完整模型並預測測試集


/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [23:41:20] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [56]:
# === 等級預測（level）===
df_level, model_level, cols_level = train_and_predict_xgb(
    y_name='level',
    smote_ratio=0.5,  # 類別分布若較平均，可用較小比例
    return_model=True,
    cv_mode='manual_kfold',
    fold_count=5,
    n_iter=30
)


🚀 開始訓練任務：level（cv_mode=manual_kfold）

🔍 開始搜尋共 30 組參數，每組做 5-Fold 驗證
------------------------------------------------------------

🧪 測試第 1/30 組參數：{'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 1, 'scale_pos_weight': 1}

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{3: 13, 1: 4, 0: 14, 2: 2}
🔹 驗證集：{0: 4, 3: 3, 1: 1, 2: 1}
👤 驗證集 player_id：[27, 20, 1, 35, 5, 7, 19, 38, 23]


/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [23:42:14] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


  ➤ Fold 1 ROC AUC: 0.6849

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{1: 4, 0: 14, 3: 13, 2: 2}
🔹 驗證集：{3: 3, 0: 4, 2: 1, 1: 1}
👤 驗證集 player_id：[4, 28, 13, 14, 8, 17, 9, 34, 6]


/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [23:44:13] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


  ➤ Fold 2 ROC AUC: 0.6239

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{0: 14, 3: 13, 1: 4, 2: 2}
🔹 驗證集：{3: 3, 0: 4, 1: 1, 2: 1}
👤 驗證集 player_id：[11, 26, 7, 30, 35, 40, 16, 24, 9]


...

/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [03:23:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


  ➤ Fold 4 ROC AUC: 0.7808

📊 每個類別在訓練與驗證集的 player 數：
🔹 訓練集：{1: 4, 0: 14, 3: 13, 2: 2}
🔹 驗證集：{3: 3, 0: 4, 1: 1, 2: 1}
👤 驗證集 player_id：[1, 18, 33, 39, 10, 3, 21, 23, 17]


/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [03:25:14] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


  ➤ Fold 5 ROC AUC: 0.7958
✅ 平均 ROC AUC: 0.7133

🏁 最佳 ROC AUC: 0.7400 對應參數：{'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.05, 'subsample': 0.6, 'colsample_bytree': 1.0, 'gamma': 1, 'scale_pos_weight': 1}

🏆 最佳 ROC AUC: 0.7400
🎯 使用最佳參數重新訓練完整模型並預測測試集


/home/barry/.conda/envs/test/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [03:26:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [61]:
# ==== 6. 合併所有預測結果 ====
submission = pd.concat([unique_ids, df_gender, df_hold, df_play, df_level], axis=1)


In [76]:
submission

,unique_id,gender,hold racket handed,play years_0,play years_1,play years_2,level_0,level_1,level_2,level_3
0,2096,0.996293,0.999230,0.408593,0.334860,0.256547,0.040960,0.013726,0.010653,0.934661
1,2096,0.998237,0.999961,0.544745,0.254326,0.200929,0.441949,0.422465,0.000911,0.134675
2,2096,0.999454,0.999978,0.436762,0.303865,0.259373,0.722389,0.070861,0.001725,0.205025
3,2096,0.999843,0.999970,0.467646,0.248203,0.284151,0.880449,0.093451,0.000837,0.025263
4,2096,0.999152,0.999957,0.534873,0.252426,0.212701,0.777577,0.137027,0.001675,0.083721
...,...,...,...,...,...,...,...,...,...,...
38009,3394,0.998616,0.126737,0.446652,0.319620,0.233728,0.150040,0.028127,0.009066,0.812767
38010,3394,0.997283,0.287645,0.429543,0.324214,0.246243,0.124291,0.009876,0.008252,0.857581
38011,3394,0.999673,0.160658,0.404595,0.347701,0.247704,0.125389,0.068996,0.005749,0.799866
38012,3394,0.998816,0.217383,0.491348,0.264207,0.244445,0.204451,0.020428,0.008709,0.766413


# 二元分類調整機率後處理

In [103]:
def aggregate_by_majority_vote_with_confidence(submission, thresholds=None):
    df = submission.copy()

    # === 1. 二分類欄位的 threshold 處理 ===
    if thresholds is None:
        thresholds = {
            'gender': 0.8,
            'hold racket handed': 0.5,
        }

    for col, th in thresholds.items():
        df[col] = (df[col] > th).astype(int)

    # === 2. 重命名多分類欄位 ===
    level_rename_map = {
        'level_0': 'level_2',
        'level_1': 'level_3',
        'level_2': 'level_4',
        'level_3': 'level_5',
    }

    df.rename(columns=level_rename_map, inplace=True)

    # === 3. groupby by unique_id 統計每個欄位平均（即票數平均） ===
    group_cols = ['gender', 'hold racket handed',
                  'play years_0', 'play years_1', 'play years_2',
                  'level_2', 'level_3', 'level_4', 'level_5']

    result = (
        df[['unique_id'] + group_cols]
        .groupby('unique_id')
        .mean()
        .reset_index()
        .sort_values(by='unique_id')
        .reset_index(drop=True)
    )

    result.to_csv("final_submission_grouped_with_fallback.csv", index=False)
    print("✅ 已輸出：final_submission_grouped_with_fallback.csv")
    return result


In [104]:
final_submission = aggregate_by_majority_vote_with_confidence(submission)

✅ 已輸出：final_submission_grouped_with_fallback.csv


In [105]:
final_submission

,unique_id,gender,hold racket handed,play years_0,play years_1,play years_2,level_2,level_3,level_4,level_5
0,1968,0.962963,1.0,0.286402,0.484965,0.228633,0.044376,0.760122,0.007270,0.188232
1,1969,1.000000,1.0,0.169251,0.662377,0.168372,0.001199,0.050972,0.000227,0.947602
2,1970,1.000000,1.0,0.190380,0.598732,0.210888,0.002188,0.102103,0.000912,0.894797
3,1971,1.000000,1.0,0.163758,0.668062,0.168180,0.001563,0.045646,0.000598,0.952193
4,1972,0.962963,1.0,0.207260,0.417121,0.375619,0.002605,0.505512,0.000920,0.490963
...,...,...,...,...,...,...,...,...,...,...
1425,3407,1.000000,1.0,0.183012,0.613738,0.203249,0.003057,0.081642,0.000577,0.914723
1426,3408,0.962963,1.0,0.203147,0.519260,0.277593,0.002017,0.317249,0.000377,0.680358
1427,3409,1.000000,1.0,0.350047,0.411956,0.237997,0.061824,0.841110,0.004760,0.092306
1428,3410,0.962963,1.0,0.175278,0.543667,0.281055,0.002617,0.772307,0.000331,0.224744


# 查看預測分布並決定如何調整多分類預測結果

In [106]:
print("📊 Gender 分類分布：")
print((final_submission['gender'] > 0.5).value_counts().rename({True: '男', False: '女'}))

print("\n📊 慣用手分類分布：")
print((final_submission['hold racket handed'] > 0.5).value_counts().rename({True: '右手', False: '左手'}))

# === play years 分布（多分類）===
print("\n📊 Play Years 分類分布（最大機率對應類別）：")
play_cols = ['play years_0', 'play years_1', 'play years_2']
play_pred_idx = final_submission[play_cols].idxmax(axis=1)  # 取最大機率對應欄位
play_pred_label = play_pred_idx.str.extract(r'play years_(\d)').astype(int)[0]
play_counts = play_pred_label.value_counts().sort_index().rename(lambda x: f"年資_{x}")
print(play_counts)

# === level 分布（多分類）===
print("\n📊 Level 分類分布（最大機率對應類別）：")
level_cols = ['level_2', 'level_3', 'level_4', 'level_5']
level_pred_idx = final_submission[level_cols].idxmax(axis=1)
level_pred_label = level_pred_idx.str.extract(r'level_(\d)').astype(int)[0]
level_counts = level_pred_label.value_counts().sort_index().rename(lambda x: f"等級_{x}")
print(level_counts)



📊 Gender 分類分布：
gender
男    1305
女     125
Name: count, dtype: int64

📊 慣用手分類分布：
hold racket handed
右手    1215
左手     215
Name: count, dtype: int64

📊 Play Years 分類分布（最大機率對應類別）：
0
年資_0      91
年資_1    1182
年資_2     157
Name: count, dtype: int64

📊 Level 分類分布（最大機率對應類別）：
0
等級_2    168
等級_3    335
等級_5    927
Name: count, dtype: int64


272 629 512；515 143 86 657

In [107]:
def print_train_y_distributions(train_df):
    print("📊 Gender 分布（1=男, 2=女）：")
    print(train_df['gender'].value_counts(normalize=True).sort_index())
    
    print("\n📊 慣用手分布（1=右手, 2=左手）：")
    print(train_df['hold racket handed'].value_counts(normalize=True).sort_index())

    print("\n📊 Play Years 分布（0=新手, 1=中等, 2=資深）：")
    print(train_df['play years'].value_counts(normalize=True).sort_index())

    print("\n📊 Level 分布（2~5）：")
    print(train_df['level'].value_counts(normalize=True).sort_index())
print_train_y_distributions(train_df)


📊 Gender 分布（1=男, 2=女）：
gender
1    0.832148
2    0.167852
Name: proportion, dtype: float64

📊 慣用手分布（1=右手, 2=左手）：
hold racket handed
1    0.812353
2    0.187647
Name: proportion, dtype: float64

📊 Play Years 分布（0=新手, 1=中等, 2=資深）：
play years
0    0.198410
1    0.443371
2    0.358219
Name: proportion, dtype: float64

📊 Level 分布（2~5）：
level
2    0.366053
3    0.102115
4    0.069826
5    0.462007
Name: proportion, dtype: float64


# 儲存預測結果

In [ ]:
# ==== 7. 輸出為 CSV ====
file_name = 'model_1_pred.csv'
final_submission.to_csv(file_name, index=False, encoding='utf-8')
print(f"✅ 輸出完成：{file_name}")

✅ 輸出完成：submission_Su20250601_4.csv
